<a href="https://colab.research.google.com/github/phanmanhtung/Natural-Language-Processing-Course/blob/master/Programming_Assignment_5_Word_Vectors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Programming Assignment 5 - Word Vectors

In this programming assignment, we will implement a distributional method for computing word similarities.

You may want to refer to the Section 20.7 "Word Similarity: Distributional Methods" in the textbook [1].

The due for the assignment is on **April 29 (Wednesday)**.

## Exercise 0: Write your name and student ID

Write your name and student id

- Name: Phan Manh Tung
- Student ID: Bi8-160

## Dataset

We use the data in the file [enwiki-tokenized-v2.txt](https://drive.google.com/file/d/1TUvn3xT3CwH1V9RhC0C4KLNCEOh7eolC/view?usp=sharing). This file contains tokenized text files sampled from Wikipedia.

First, we will download the text file.

In [0]:
!rm -f enwiki-tokenized-v2.txt

from google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id="1TUvn3xT3CwH1V9RhC0C4KLNCEOh7eolC",
                                    dest_path="./enwiki-tokenized-v2.txt",
                                    unzip=False)

In [0]:
!head -5 enwiki-tokenized-v2.txt

In May 1963 Minter suffered serious injuries when racing a Norton at Brands Hatch , and his place in the team for the TT races was taken by Phil Read , who came third to second-place team mate Hartle . The team only raced for one season in selected races .
The city of Kozhikode is north of the state capital Thiruvananthapuram . It is located at approximately . It has an elevation of 1 metre ( 3 ft ) along the coast with the city 's eastern edges rising to at least 15 metres , with a sandy coastal belt and a lateritic midland . The city has a long shoreline and small hills dot the terrain in the eastern and central regions . To the city 's west is the Laccadive Sea and from approximately to the east rises the Sahyadri Mountains .
Fives is a minor sport , with courts scattered across the country , and a limited presence in a handful of private schools .
Test of highest and best use .
Titus , a slave to Mrs. John Cabot of Salem , established a business and successfully recruited blacks as

## Exercise 1: Extracting Context (10 points)

In this exercise, you will extract context of all words in the file `enwiki-tokenized-v2.txt`.

For each word *t*, you will extract surrounding words in the window size 5. They are 5 words occuring before/after of the word.

Let's consider the example sentence as bellow.

    He was born William McKinley Randle Jr. in Detroit , Michigan .

The context of the word "McKinley" include words {"He", "was", "born", "William", "Randle", "Jr.", "in", "Detroit", ","}. Words "Michigan" and "." are not included in the context of the word "McKinley".

Write the context of words into the file "enwiki-tokenized-context.txt". Each line contains the word *t* and a word *c* in its context. Two words in a line are separated by a tab character.

For the word William in the above sentence, we will write following lines.

```
William    He
William    was
William    born
William    Randle
William    Jr.
William    in
William    Detroit
William    ,
```

Write the code to do the job.

In [0]:
# Testing code

with open("enwiki-tokenized-v2.txt") as f:
  time = 0
  for line in f:
    words = line.split(" ")
    for word in words:
      if (words.index(word)-5) <= 0:
        for before in words[0:(words.index(word))]:
          print(word, "\t", before, "\n")
        for after in words[(words.index(word)+1):(words.index(word)+6)]:  
          print(word, "\t", after, "\n")
      else:
        for before in words[(words.index(word)-5):(words.index(word))]:
          print(word, "\t", before, "\n")
        for after in words[(words.index(word)+1):(words.index(word)+6)]:  
          print(word, "\t", after, "\n")
    time+=1
    if time==3:
      break

In [0]:
# The main writing file function

with open("enwiki-tokenized-v2.txt") as f:
  for line in f:
    words = line.split(" ")
    with open("enwiki-tokenized-context.txt", "w") as fo:
      for word in words:
        if (words.index(word)-5) <= 0:
          for before in words[0:(words.index(word))]:
            fo.write(word + "\t" + before + "\n")
          for after in words[(words.index(word)+1):(words.index(word)+6)]:  
            fo.write(word + "\t" + after + "\n")
        else:
          for before in words[(words.index(word)-5):(words.index(word))]:
            fo.write(word + "\t" + before + "\n")
          for after in words[(words.index(word)+1):(words.index(word)+6)]:  
            fo.write(word + "\t" + after + "\n")

In [0]:
# The main writing file function

with open("enwiki-tokenized-v2.txt") as f:
  for line in f:
    words = line.split(" ")
    with open("enwiki-tokenized-context.txt", "w") as fo:
      for word in words:
        if (words.index(word)-5) <= 0:
          for before in words[0:(words.index(word))]:
            fo.write("%s  %s\n" % (word,before))
          for after in words[(words.index(word)+1):(words.index(word)+6)]:  
            fo.write("%s  %s\n" % (word,after))
        else:
          for before in words[(words.index(word)-5):(words.index(word))]:
            fo.write("%s  %s\n" % (word,before))
          for after in words[(words.index(word)+1):(words.index(word)+6)]:  
            fo.write("%s  %s\n" % (word,after))

In [0]:
!head -20 enwiki-tokenized-context.txt

In  the
In  British
In  period
In  it
In  was
the  In
the  British
the  period
the  it
the  was
the  a
British  In
British  the
British  period
British  it
British  was
British  a
British  Royal
period  In
period  the


## Exercise 2: Computing Frequency of Words in Contexts (20 points)

In this exercise, you will write code to calculate frequencies of words in contexts you extracted in the Exercise 1 (File "enwiki-tokenized-context.txt"). Specifically, you will compute following values:

- *f*(*t*,*c*): The number of times word *t* and context word *c* co-occur.
- *f*(*t*,\*): The number of time word *t* occurs.
- *f*(\*,*c*): The number of time the context word *c* occurs.
- *N*: The total number of times words and their context words occur.

In [0]:
from collections import defaultdict

f_tc = defaultdict(int)
f_t = defaultdict(int)
f_c = defaultdict(int)
N = 0

with open("enwiki-tokenized-context.txt", "r") as f:
  for line in f:
    line = line.strip()
    text = line.split("  ")
    word = text[0]
    context = text[1]
    f_tc[word + '\t' + context] += 1
    f_t[word] += 1
    f_c[context] += 1
    N += 1

print(f_tc)
print(f_t)
print(f_c)
print(N)



IndexError: ignored

In [0]:
with open("enwiki-tokenized-context.txt") as f:
  for line in f:
    a = line.split("  ")
    b = a[1]
    print(b)

the

British

period

it

was

In

British

period

it

was

a

In

the

period

it

was

a

Royal

In

the

British

it

was

a

Royal

Air

In

the

British

period

was

a

Royal

Air

Force

In

the

British

period

it

a

Royal

Air

Force

(

the

British

period

it

was

Royal

Air

Force

(

RAF

British

period

it

was

a

Air

Force

(

RAF

)

period

it

was

a

Royal

Force

(

RAF

)

station

it

was

a

Royal

Air

(

RAF

)

station

,

was

a

Royal

Air

Force

RAF

)

station

,

an

a

Royal

Air

Force

(

)

station

,

an

airfield

Royal

Air

Force

(

RAF

station

,

an

airfield

.

Air

Force

(

RAF

)

,

an

airfield

.

It

Force

(

RAF

)

station

an

airfield

.

It

played

(

RAF

)

station

,

airfield

.

It

played

host

RAF

)

station

,

an

.

It

played

host

to

)

station

,

an

airfield

It

played

host

to

airlines

station

,

an

airfield

.

played

host

to

airlines

,

,

an

airfield

.

It

host

to

airlines

,

and


IndexError: ignored

## Exercise 3: Computing Term-Context Matrix (20 points)

In this exercise, you will use the output of Exercise 2 to compute the term/context matrix $X$. Rows of matrix represent words, and columns represent words in contexts. Values $X_{tc}$ of term/context matrix are defined as follows.

- If $f(t,c) \geq 10$, then $X_{tc}=\text{PPMI}(t,c)=\text{max}\left(\log \frac{N\times f(t,c)}{f(t,*)\times f(*,c)}, 0\right)$
- If $f(t,c) < 10$, then $X_{tc}=0$

Here $\text{PPMI}(t, c)$ denotes Pointwise Mutual Information. Note that because the matrix size $X$ is very large, saving all matrix values to memory is not possible. You can use sparse matrix storage technique with the note that most of the values of the elements in $X$ are equal to 0.

Hint: You can use [sparse](https://docs.scipy.org/doc/scipy/reference/sparse.html) matrix in scipy.

In [0]:

"""Write your code to compute term-context matrix here
"""
pass

## Exercise 4: Dimention Reduction with SVD (20 points)

Use the SVD algorithm for the matrix obtained in Exercise 3 to reduce the number of data dimensions so that the resulting vector words have a dimension of 300.

You can use [SVD](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html) implementation in sklearn.

The implementation is something like as follows.
```
import sklearn.decomposition

# matrix_x is the original sparse matrix

clf = sklearn.decomposition.TruncatedSVD(300)
matrix_x300 = clf.fit_transform(matrix_x)
```

In [0]:
"""TODO: Write your code for dimension reduction here
"""
pass

## Exercise 5: Getting Word Vectors (10 points)

Use word vectors obtained in Exercise 4, displaying the vector for the word "United_States".

In [0]:
"""Write your code for getting word vector of a word here
"""
pass

## Exercise 6: Calculating Word Similarity (20 points)

Using the word vectors obtained in the Exercise 5, calculate cosine similarity for two words "United_States" and "U.S".

In [0]:
"""TODO: Write your code for calculating word similarity here
"""
pass

## References

1. Jurafsky et al. 2007. Speech and Language Processing (Second Edition).